## Clustering Planare Winkel

In [ ]:
from config import *
trna = load_data('_cleaned_L2_trna')
agg_all = load_agg_df()

Zum Clustering wir die Funktion `cluster.clustering.do_cluster` verwendet, die viele optionale Parameter besitzt. Wir setzen hier:

In [ ]:
kwargs = {}  # Parameter dict with entry for each loop_type

for way in range(3, 9):
    kwargs[way] = dict(
        data=load_agg_df(way=way), # Aggregierte Multiloops
        features=[f'planar_{i}_median' for i in range(1, way+1)],  # Median
        dim=way,                   # Cluster in all dimensions of loop_type
        alg="agglomerative",       # agglomerative|k_means|meanshift|spectral
        scaler=None,               # None|MinMax|Standard -> for angles None
        s=2,                       # Marker size for plots
        save=True,                 # Whether to save result plots
        legend=False,              # Whether to draw legend
    )

75 Cluster zeigte für alle Looptypen ordentliche Ergebnisse. Da wir ohnehin nachselektieren müssen, ist die genaue Zahl nicht komplett entscheidend. \
Wir probieren aber für alle loop_typen bis 8 trotzdem einige aus um den Einfluss abschätzen zu können.

Im Folgenden sind alle Ergebnisse dargestellt. Instruktiver ist das ganze aber nachdem analyse_cluster ausgeführt wurde. \
Damit werden cluster mit weniger als 10 Sequenzen entfernt und die übrigen in zwei Schritten weiter untersucht:
1) Die Größe des Clusters wird mittels threshold reduziert. Metrik hierzu ist die mittlere Abweichung der Winkel einer Sequenz vom Median des Clusters.
2) Charakteristika der resultierenden Cluster werden aus den Rohdaten gesammelt und in einem Bild pro Cluster dargestellt.

Als Grenze in Schritt 1 (threshold) werden 2°, 4°, 6° und 8° gesetzt, was jeweils unterschiedlich große Cluster aus den initialen agglomerativen clustern erzeugt. \
Alle so entstehenden Ergebnisse sind unter results/cluster_ids zu finden. Jedes Cluster mit mehr als 10 Sequenzen hat ein Bild mit dem Sub-Pfad: 

< Anzahl initialer agglomerativer Cluster (n_cluster) > / \
< Loop-Typ des Clusters (Anzahl Stems) > / \
< Untere Grenze für Punktdichte im Cluster > / \
< {n_cluster}-{cluster_idx}_dev{threshold}.png > 

Dabei wird die Punktdichte pro Winkelvolumen bezogen auf die Streuung der Punkte berechnet gemäß

$ \text{density} = N_c \cdot \left( \frac{1}{n} \cdot \sum_{i=0}^{n} \sqrt{ \frac{1}{N_c^2} \sum_{j=0}^{N_c} (\varphi_{i, j} - \widetilde{\varphi_i})^2}) \right) ^ {-n} $

mit $N_c$ als Zahl der Sequencen im Cluster, $n$ als loop-type (Anzahl Stems), $\varphi_{i, j}$ als i-tem planaren Winkel der Sequenz j und $\widetilde{\varphi_i}$ als Median dieses Winkels im Cluster. \
Die Formel beschreibt also die Dichte der Punkte im Cluster bezogen auf das Volumen des Clusters im n-dimensionalen Featureraum, wobei n der Anzahl der Stems entspricht, da jeder von diesen einen Winkel beisteuert. \
Als Maß für die Größe des Volumens, wird entlang jedes Winkels die Standardabweichung der Werte dieses Features für alle Sequenzen gesetzt. \
Je höher die Dichte, desto unwahrscheinlicher ist eine zufällige Agglomeration von Sequenzen in diesem Winkelbereich.

In den Ordern sind zudem die Bilder von unten, die Zugehörigkeit der Sequenzen zu den Clustern und csv Dateien für die Cluster zu finden.

Die png Dateien im Pfad wie oben beschrieben liefern einen Gesamtüberblick über Charakteristika der Cluster.

In [ ]:
for n in [25, 50, 75, 100, 150, 200]:
    for way in range(3, 9):
        path = f'cluster_ids/n_{n}/way{way}/all_cluster_way{way}'
        do_cluster(n_clusters=(n, ), scatterplot_path=path, **kwargs[way])
        plt.show()